In [3]:
import os
import sys
custom_paths = ['../src/', '../cifar-10.1-pytorch/', '../data-copying/', '../PyTorch_CIFAR10/']
for path in custom_paths:
    if os.path.abspath(path) not in sys.path:
        sys.path.insert(0, os.path.abspath(path))
        print(f"Added {os.path.abspath(path)} to path")

Added /nfs/home/cybai/PyTorch-StudioGAN/src to path
Added /nfs/home/cybai/PyTorch-StudioGAN/cifar-10.1-pytorch to path
Added /nfs/home/cybai/PyTorch-StudioGAN/data-copying to path
Added /nfs/home/cybai/PyTorch-StudioGAN/PyTorch_CIFAR10 to path


In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import functools
from importlib import reload
from matplotlib import pyplot as plt
import scipy
from scipy import stats
from scipy.special import softmax
import os
import ast
import glob
import pickle

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans 

import torch
from torch.nn import Parameter as P
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# import custom

import cifar10_models
reload(cifar10_models)

import cifar10_1
reload(cifar10_1)

from data_utils import cifair
reload(cifair)

from metrics.feat_extract_models import alexnet, inception, vgg
reload(alexnet)
reload(inception)
reload(vgg)

from utils import load_checkpoint
reload(load_checkpoint)

from metrics.FID import calculate_frechet_distance

import misc
reload(misc)

import data_copying_tests
reload(data_copying_tests)

<module 'data_copying_tests' from '/nfs/home/cybai/PyTorch-StudioGAN/data-copying/data_copying_tests.py'>

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = 'cuda'
torch.backends.cudnn.benchmark = True

In [6]:
proj_model = inception.WrappedInception3().eval().to(device)

##### trainable parameters

#####


In [1]:
sagan_runs = {
    0.00: ['SAGAN-train-2021_03_04_15_20_57', 'SAGAN-train-2021_03_09_00_54_36', 'SAGAN-train-2021_03_10_10_00_29', 'SAGAN-train-2021_03_17_13_42_52'],
    0.10: ['SAGAN-train-2021_03_06_00_44_13', 'SAGAN-train-2021_03_09_00_57_09', 'SAGAN-train-2021_03_11_10_38_54', 'SAGAN-train-2021_03_18_13_59_54'],
    0.12: ['SAGAN-train-2021_03_06_00_46_59', 'SAGAN-train-2021_03_09_01_00_30', 'SAGAN-train-2021_03_12_11_16_43', 'SAGAN-train-2021_03_28_13_20_51'],
    0.13: ['SAGAN-train-2021_03_06_00_48_39', 'SAGAN-train-2021_03_09_02_08_08', 'SAGAN-train-2021_03_13_11_47_27', 'SAGAN-train-2021_03_28_13_21_53'],
    0.14: ['SAGAN-train-2021_03_06_00_50_43', 'SAGAN-train-2021_03_27_11_48_10', 'SAGAN-train-2021_03_14_12_23_07', 'SAGAN-train-2021_03_28_13_30_23'],
    0.15: ['SAGAN-train-2021_03_06_00_54_20', 'SAGAN-train-2021_03_27_11_51_51', 'SAGAN-train-2021_03_15_12_50_35', 'SAGAN-train-2021_04_03_16_07_22'],
    0.16: ['SAGAN-train-2021_03_06_00_57_47', 'SAGAN-train-2021_03_27_11_53_44', 'SAGAN-train-2021_03_16_13_20_09', 'SAGAN-train-2021_04_03_16_10_52']
}

In [2]:
diffaug_runs = {
    0.00: ['DiffAugGAN(P)-train-2021_03_06_20_15_56', 'DiffAugGAN(P)-train-2021_03_10_09_58_03', 'DiffAugGAN(P)-train-2021_03_25_22_02_38', 'DiffAugGAN(P)-train-2021_04_13_21_24_13'],
    0.10: ['DiffAugGAN(P)-train-2021_03_06_20_19_47', 'DiffAugGAN(P)-train-2021_03_12_14_55_48', 'DiffAugGAN(P)-train-2021_03_28_03_17_16', 'DiffAugGAN(P)-train-2021_04_13_21_24_52'],
    0.12: ['DiffAugGAN(P)-train-2021_03_06_20_22_27', 'DiffAugGAN(P)-train-2021_03_14_19_57_50', 'DiffAugGAN(P)-train-2021_03_30_08_38_45', 'DiffAugGAN(P)-train-2021_04_17_10_54_23'],
    0.13: ['DiffAugGAN(P)-train-2021_03_06_20_25_10', 'DiffAugGAN(P)-train-2021_03_17_01_39_09', 'DiffAugGAN(P)-train-2021_04_01_13_59_14', 'DiffAugGAN(P)-train-2021_04_17_10_59_40'],
    0.14: ['DiffAugGAN(P)-train-2021_03_06_20_26_45', 'DiffAugGAN(P)-train-2021_03_19_06_43_02', 'DiffAugGAN(P)-train-2021_04_03_19_03_34', 'DiffAugGAN(P)-train-2021_04_19_22_56_11'],
    0.15: ['DiffAugGAN(P)-train-2021_03_08_14_46_12', 'DiffAugGAN(P)-train-2021_03_21_11_38_59', 'DiffAugGAN(P)-train-2021_04_06_00_25_51', 'DiffAugGAN(P)-train-2021_04_06_13_47_56'],
    0.16: ['DiffAugGAN(P)-train-2021_03_27_11_52_47', 'DiffAugGAN(P)-train-2021_03_23_16_50_02', 'DiffAugGAN(P)-train-2021_04_08_05_55_17', 'DiffAugGAN(P)-train-2021_04_03_16_19_34'],
}

In [12]:
for runs in [sagan_runs, diffaug_runs]:
    for runnames in runs.values():
        for runname in runnames:
            sampler = misc.wrapper_best(runname, dset_name='CIFAR10', device=device)
            misc.build_cache(runname, sampler, cache_dir='/tmp2/cybai/studiogan/cache', 
                             conditional=True, proj_model=proj_model, verbose=True)

cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_04_15_20_57/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_09_00_54_36/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_10_10_00_29/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_17_13_42_52/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_09_00_57_09/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_11_10_38_54/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_18_13_59_54/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_06_00_46_59/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAG